In [ ]:
import time
import serial
import sys
import glob
import serial
import numpy as np

In [ ]:
#https://stackoverflow.com/questions/12090503/listing-available-com-ports-with-python
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [ ]:
ports = serial_ports()
print('Available serial ports: ')
for p in ports:
    print(ports.index(p), end=':')
    print(p)

In [ ]:
portindex = 1

In [ ]:
try:
    myport = serial.Serial(
        port=ports[portindex],
        baudrate=250000,
        timeout=1.0,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.EIGHTBITS
    )
    print('Sucessfully connected to port '+myport.port)
except (OSError, serial.SerialException):
    print('ERROR: ', end='')
    if myport.is_open:
        print('Port already open')
    elif myport.port not in portlist:
        print('Port '+myport.port+' not found')
    else:
        print('Could not connect to port')
    pass

In [ ]:
commandPrompt='Command: '

exitTerminal='exit'
getHelp='help'

lineTerminator='\r\n'
shortcuts = {
    'home':'G28',
    'deploy':'M401',
    'stow':'M402',
    'fanon':'M106 P0 S255',
    'fanoff':'M107',
    'position':'M114'
}
def probe_point():
    print('probe point macro')
    sendgcode('m105')
def destroy():
    return
macros = {
    'probe point':probe_point,
    'destroy machine':destroy
}

In [ ]:
def parse_coordinates(printeroutput):
    coords = printeroutput.rstrip().split()
    del coords[3:]
    parsedcoords = list(map(float,list(map(lambda coord: (''.join(filter(lambda char: char not in ['X', 'Y', 'Z', ':'], coord))), coords))))
    print(parsedcoords)
    
    
def sendgcode(gcode):
    raw = gcode.upper()
    print('SEND: ' + raw); #remove trailing whitespace and print
    sInput = (raw + lineTerminator) #append terminator string
    bInput = sInput.encode() #convert string to bytes
    myport.write(bInput) #send the command

In [ ]:
testEcho = "X:280.1234 Y:0.1234 Z:55.1234 E:0.0000 Count X:22400 Y:0 Z:22000"
parse_coordinates(testEcho)

In [ ]:
print ('Enter your commands below.\r\nType '+"'"+exitTerminal+"'"+' to leave the interactive prompt, type '+"'"+getHelp+"'"+' to see available shorcuts')
cansend=True;
while True:
    if(cansend):
        userinput=''
        userinput = input(commandPrompt)
        if userinput == exitTerminal:
            break
        elif userinput == getHelp:
            print(shortcuts)
            print(macros)
        elif userinput in shortcuts:
            cansend=False
            sendgcode(shortcuts[userinput])
        elif userinput in macros:
            cansend=False
            cansend=macros[userinput]()
        else:
            cansend=False
            sendgcode(userinput)

    time.sleep(0.1) # wait a brief moment before reading the serial buffer
    
    lines = list(map(lambda l: l.decode().rstrip(), myport.readlines()))
    for line in lines:
        print('RECV: '+ line)
        if(line[:2] == 'ok'):
            cansend=True
            myport.reset_input_buffer()
